### CS4423 - Networks
Prof. Götz Pfeiffer<br />
School of Mathematics, Statistics and Applied Mathematics<br />
NUI Galway

# Lecture 5: Paths, Trees and Breadth First Search

Sequences of interconnected edges in a  graph are called **paths**,
leading to notions of **connectivity** and **distance**.
A **tree** is a particularly useful kind of connected graph,
that is frequently used as a data structure in Computer Science.

Many questions on networks concerning distance and connectivity can
be answered by a versatile strategy called **Breadth First Search (BFS)**
which effectively grows a **spanning tree** of the underlying graph.

We will study **trees** and some classical algorithms for tree traversal.
A network has the structure of a tree when it is of a hierarchical nature,
like an [ancestry chart](https://en.wikipedia.org/wiki/Pedigree_chart)
or a **river network**.

![A River Network](images/rivers.jpg)

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

## Paths

The fundamental notion of *connectivity* in a network is closely
related to the notion of *paths* in a graph.

<div class="alert alert-danger">
<b>Definitions.</b>  
    <ul>
        <li> 
            A <b>path</b> in a graph $G = (X, E)$ is a sequence of nodes, 
            where any pair of consecutive nodes in the sequence is (linked by)
            an edge in $E$.
        </li>
        <li>
            Such a path can have repeated nodes.  If it doesn't, the path is called a <b>simple path</b>.
        </li>
        <li>
            The <b>length</b> of a path is the number of edges it involves
            (that is the number of nodes minus $1$).
        </li>
        <li>
            At each vertex $x \in X$, there is a unique path of length $0$, 
            the <b>empty path</b>, consisting of vertex $x$ only.
        </li>
        <li>
            A <b>cycle</b> is a path of length at least $3$ that is a simple path,
            except for the first and the last node being the same.
        </li>
    </ul>
</div>

In [ ]:
nodes = 'ABCDEFGHIJKLM'

In [ ]:
G = nx.Graph()

In [ ]:
G.add_nodes_from(nodes)

In [ ]:
list(G.nodes())

In [ ]:
nodes = 'ABCDEFGHIJKLM'
edges = [
    'AB', 'CE', 'FG', 'FH', 'GI', 'GJ', 'HJ', 'HL', 'HM', 
    'IK', 'JK', 'KL', 'LM'
]
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [ ]:
opts = { "with_labels": True, "node_color": 'y' }
nx.draw(G, **opts)

* $(F, G, I)$ is a path in the graph above, and $(H, J, K, L, H)$ is a cycle.

* A cycle in a simple graph provides, for any two nodes on that
cycle, (at least) two different paths from one to the other.

* Note that each edge (and node) of the 1970 Internet graph belongs to
a cycle.  This makes the other way around the cycle an alternative
route in case one of the edges should fail.

* (In a *directed* network, paths are directed, too.
A path from a vertex $x$ to a vertex $y$ is
a sequence of vertices $x = x_0, x_1, \dots, x_k = y$
such that, for any $i = 1, \dots, k$, there is
an edge from $x_{i-1}$ to $x_i$ in the graph.)



## Connected Components

Communication and transportation networks tend to be connected, as
this is their main purpose: to connect.

<div class="alert alert-danger">
<b>Definition.</b> 
    <ul>
        <li>A simple graph is <b>connected</b> if, for
every pair of nodes, there is a path between them.
        </li>
        <li>
If a graph is not connected, it naturally breaks into pieces,
its <b>connected components</b>.
        </li>
    </ul>
</div>

* The connected components of the graph below are the
node sets $\{A, B\}$, $\{C, E\}$, $\{D\}$, and $\{F,G,H,I,J,K,L,M\}$.
* Note that a component can consist of a single node only.

In [ ]:
list(nx.connected_components(G))

**Note.** 
The relation 'there is a **path** from $x$ to $y$ on the node set $X$ of a
graph is the **transitive closure** of the graph relation 'there is an
**edge** between $x$ and $y$'.  It is 

* **reflexive** (as each node $x$ is
connected to itself by the zero length path starting and ending at
$x$), 

* **symmetric** (as a path from $x$ to $y$ can be used backwards as
a path from $y$ to $x$), 

* and **transitive** (as a path from $x$ to $y$ and
a path from $y$ to $z$ together make up a path from $x$ to $z$), 

hence
an **equivalence relation**.
The connected components of the graph are
the parts (equivalence classes) of the corresponding **partition** of $X$.

##  Trees

* A graph is called **acyclic** if it does not contain any cycles.

<div class="alert alert-danger">
    <ul>
        <li>
    A <b>tree</b> is a (simple) graph that is <b>connected</b> and <b>acyclic</b>.
        </li>
        <li>
            A <b>forest</b> is a graph whose connected components are all trees.
        </li>
    </ul>         
</div>

In other words, between any two vertices in a tree there is **exactly one simple path**.

Trees can be characterized in many different ways.

<div class="alert alert-warning">

**Theorem.**  Let $G = (X, E)$ be a (simple) graph of order $n = |X|$
and size $m = |E|$.
Then the following are equivalent:

* $G$ is a tree (i.e. acyclic and connected);

* $G$ is connected and $m = n-1$;

* $G$ is a minimally connected graph (i.e., removing any edge will disconnect $G$);

* $G$ is acyclic and $m = n-1$;

* $G$ is a maximally acyclic graph (i.e., adding any edge will introduce a cycle in $G$).
</div>

## Ordered and Rooted Trees

In some applications it is useful to specify additional attributes of a tree.

* A **rooted tree** is a tree in which one vertex has been designated the **root**.

* In a rooted tree, except for the root itself, each vertex $x$ has a unique **parent**:
  the unique neighbor that lies on the unique simple path from $x$ to the root.
  
* The **children** of vertex $x$ are all thie nodes it is the parent of.

* In an **ordered rooted tree** an explicit order is specified on the children of each node,
corresponding to an embedding ot the tree in the plane.


Algebraic formulas, computer programs can conveniently be represented by ORTs known as [**syntax trees**]( https://en.wikipedia.org/wiki/Abstract_syntax_tree).

## Random Trees

<div class="alert alert-danger">
    <b>Theorem (Cayley's Formula).</b>
    There are exactly $n^{n-2}$ distinct (labelled) trees on the $n$-vertex set 
    $X = \{0, 1, 2, \dots, n-1\}$, if $n > 1$.
</div>

There is one (trivial) tree on $1$ vertex, but there is no tree on $0$ vertices.  Some values for $n > 1$:
    

In [ ]:
domain = range(2, 10)
pd.DataFrame([n**(n-2) for n in domain], index=domain, columns=[""], dtype=int).transpose()

**Proof.** Each tree on $X = \{0, 1, 2, \dots, n-1\}$ corresponds to a unique sequence of
$n-2$ elements of $X$, its [**Prüfer Code**](https://en.wikipedia.org/wiki/Pr%C3%BCfer_sequence).

In [ ]:
n = 20
T = nx.random_tree(n)

In [ ]:
opts = { "with_labels": True, "node_color": 'y' }
nx.draw(T, **opts)

In [ ]:
degrees = [T.degree(x) for x in T]
degrees

How to determine the Prüfer code of a tree $T$ (destructively):

* Find the smallest leaf $x$
* Record the label $y$ of its unique neighbor
* Remove $x$ (and the edge $x - y$) from $T$
* Repeat until $T$ has only $2$ nodes left.

In [ ]:
def pruefer_node(tree):
    for x in tree:
        if tree.degree(x) == 1:
            for y in tree[x]:
                tree.remove_node(x)
                return y

In [ ]:
code = [pruefer_node(T) for k in range(n-2)]
code

This process destroys the tree `T` almost completely.

In [ ]:
print(T.nodes())
print(T.edges())

In [ ]:
def pruefer_seq(tree):
    return [pruefer_node(tree) for k in range(tree.order() - 2)]

Luckily, the tree can be reconstructed from its Prüfer code.

**Fact:** The degree of node $x$ is $1$ plus the number of entries $x$ in the Prüfer sequence of $T$.

In [ ]:
ddd = [1 for k in range(n)]
for k in code:
    ddd[k] += 1
ddd

In [ ]:
degrees

In [ ]:
ddd == degrees

How to restore the tree from its Prüfer code:

* Start with a graph with vertex set $X = \{0, 1, 2, \dots, n-1\}$ (and no edges yet).
* Compute the desired node degrees (defects) from the code.
* For each node $y$ in the code find the smallest degree $1$ node $x$ and
add the edge $x - y$, then decrease the defects of both $x$ and $y$ by $1$.
* Finally, connect the remaining $2$ nodes of defect $1$ by an edge.

In [ ]:
T = nx.Graph()
T.add_nodes_from(range(n))
nx.draw(T, **opts)

In [ ]:
# repeat n-2 times:
print(code)
y = code.pop(0)
print(ddd)
x = ddd.index(1)
print("Added edge", x, "--", y)
T.add_edge(x, y)
ddd[x] -= 1
ddd[y] -= 1
nx.draw(T, **opts)
print(ddd)

In [ ]:
e = [x for x in range(n) if ddd[x] == 1]
T.add_edge(*e)
print(e)
nx.draw(T, **opts)

In [ ]:
def tree_pruefer(code):

    # initialize graph a defects
    tree = nx.Graph()
    tree.add_nodes_from(range(len(code)+2))
    defects = [1 for x in tree]
    for y in code:
        defects[y] += 1
        
    # add edges
    for y in code:
        for x in tree:
            if defects[x] == 1:
                tree.add_edge(x, y)
                for z in (x, y):
                    defects[z] -= 1
                break
                
    # final edge
    e = [x for x in tree if defects[x] == 1]
    tree.add_edge(*e)
    
    return tree

* We can now construct a random tree on $n$ nodes from a random Prüfer code of length $n-2$.

In [ ]:
from random import choice

In [ ]:
code = [choice(range(n)) for k in range(n-2)]
code

In [ ]:
tree = tree_pruefer(code)
nx.draw(tree, **opts)

In [ ]:
def random_tree(n):
    code = [choice(range(n)) for k in range(n-2)]
    return tree_pruefer(code)

## Depth First Search

[DFS](https://en.wikipedia.org/wiki/Depth-first_search)
and [BFS]()

* DFS($x$, visit):
* &nbsp; $S \gets (x)$
* &nbsp; while $S \neq \emptyset$:
* &nbsp; &nbsp; $y \gets S$.pop() 
* &nbsp; &nbsp; visit($y$) 
* &nbsp; &nbsp; $S$.push($y$.next)

In [ ]:
T = random_tree(20)
nx.draw(T, **opts)

In [ ]:
TT = T.copy()
x = 0
stack = [x]
while len(stack) > 0:
    y = stack.pop()
    stack.extend(TT[y])
    TT.remove_node(y)
    print(y, stack)


## Breadth First Search

* BFS($x$, visit):
* &nbsp; $Q \gets (x)$
* &nbsp; while $Q \neq \emptyset$:
* &nbsp; &nbsp; $y \gets Q$.pop() 
* &nbsp; &nbsp; visit($y$) 
* &nbsp; &nbsp; $Q$.push($y$.next)

In [ ]:
TT = T.copy()
x = 0
queue = [x]
while len(queue) > 0:
    y = queue.pop(0)
    queue.extend(TT[y])
    TT.remove_node(y)
    print(y, queue)

In [ ]:
nx.draw(T, **opts)

##  Attributes

Attributes such as weights, labels, colors, or whatever `python` object you like, can be attached to graphs, nodes, or edges.

Each graph, node, and edge can hold key/value attribute pairs in an associated attribute dictionary (the keys must be hashable). By default these are empty, but attributes can be added or changed using `add_edge`, `add_node` or direct manipulation of the attribute dictionaries named `G.graph`, `G.node` and `G.edge` for a graph G.

### Graph Attributes

* Assign graph attributes when creating a new graph

In [ ]:
G = nx.Graph(day="Wednesday")
G.graph

* Or you can modify attributes later

In [ ]:
G.graph['day'] = 'Monday'
G.graph

### Node Attributes

Add node attributes using `add_node()`, `add_nodes_from()` or `G.node`

In [ ]:
G.add_node(1, time='5pm')
G.add_nodes_from([3, 5], time='2pm')

In [ ]:
G.nodes[1]

In [ ]:
G.nodes[5]

In [ ]:
G.nodes[1]['room'] = 714

In [ ]:
list(G.nodes(data=True))

In [ ]:
list(G.nodes.data())

* Note that adding a node to `G.nodes` does not add it to the graph, use `G.add_node()` to add new nodes.

### Edge Attributes

* Add edge attributes using `add_edge()`, `add_edges_from()` or subscript notation.

In [ ]:
G.add_edge(1, 2, weight=4.7)
G.add_edges_from([(3, 4), (4, 5)], color='red')
G.add_edges_from([(1, 2, {'color': 'blue'}), (2, 3, {'weight': 8})])
G[1][2]['weight'] = 4.7

In [ ]:
list(G.edges(data=True))

In [ ]:
G[1][2]

## Adjacency Matrix

A useful, algebraic, way to represent a graph is given by its __adjacency matrix__.  This is square matrix $A$, with rows and columns corresponding to the vertices of the graph, and an entry $1$ or $0$ in row $i$, column $j$, if
the corresponding vertices are joined by an edge or not.
The edge $AB$ in the example above this gives an entry $1$
in row 1 (corresponding to vertex $A$) and column 2 (corresponding to
vertex $B$.  And another entry $1$ in row 2 column 1.  The full adjacency matrix
of the above graph is as follows.

$A = \left[
\begin{array}{cccc}
0&1&0&0\\
1&0&1&1\\
0&1&0&1\\
0&1&1&0
\end{array}
\right]$

Note that $A = (a_{ij})$, like every adjacency matrix of a simple
graph, is _symmetric_ (about the diagonal): $a_{ij} = a_{ji}$ for all
$i, j$.  Also, all diagonal entries are $0$.

##  Directed Graphs, Multigraphs

In this simple setting, two nodes are either connected or not (that
is, there are no multiple links) and if node $x$ is connected to node
$y$, then it is also true that $y$ is connected to $x$: the edges set
up a _symmetric_ relationship between nodes.  If we want to express
more complex real world relationships, we can make use of the more
complex structure of _directed_ graphs, where edges have directions
(drawn as arrows), possibly pointing back to the node they came from
(forming loops, or self-edges), or even with multiple edges between
the same two nodes.  A graph that has multiple edges between nodes is
usually called a _multigraph_.

![graph2]

The adjacency matrix $A$ of a directed graph has an entry $a_{ij} = 1$
whenever there is an edge from vertex $i$ to vertex $j$ (in that
order!)  in the graph, and entries $0$ in all other positions.  For
example,

$A = \left[
\begin{array}{cccc}
0&1&0&0\\
0&0&1&1\\
0&0&0&1\\
0&0&0&0
\end{array}
\right]$

A simple graph can be regarded as a directed graph by reading every
undirected edge as two directed edges between the end points,
one going forward and one coming back.

The adjacency matrix is capable of storing additional information,
like weights or multiplicities of edges or loops, if that is required.
We will soon see, how algebraic operations like matrix multiplication
can be used to obtain information about a network from its adjacency matrix.

## Degree

The **degree** of a vertex $x$ in a simple graph is the number of
vertices it is connected to in the graph (it's number of **friends**).
The degree can serve as a (simple) measure of the importance of a node
in a network.  The **average degree** of the nodes in a network depends
(only) on the number $n$ of nodes, and the number $m$ of edges in the
network.

Writing $k_i$ for the degree of vertex $x_i$, this number
easily be determined from the adjacency matrix $A$ as the number of
entries $1$ in row $i$ (or in column $i$):

$k_i = \sum_j a_{ij} = \sum_j a_{ji}$.

As every edge contributes to the degree of $2$ nodes, the sum of all degrees
equals twice the number of edges:

$\sum_i k_i = 2m$,

whence the _average degree_ is

$c = \frac1n \sum_i k_i = \frac{2m}{n}$.

## Graphs are Relations

Recall the following definitions.

<div class="note" markdown="1">
**Definition.**  A __relation__ from a set $X$ to a set $Y$ is (nothing but) a subset
$R$ of the Cartesian product $X \times Y = \\{(x, y) :  x \in X,\, y \in Y \\}$.
We say that $x \in X$ is __$R$-related__ to $y \in Y$ whenever $(x, y) \in R$
and then write $x R y$.
</div>

The adjacency matrix of a relation $R \subseteq X \times Y$
is the matrix with one row for each element $x \in X$ and one column for each
element $y \in Y$, and it has an entry $1$ in row $x$ and column $y$
if $x R y$, and entries $0$ otherwise.

In many cases, $Y = X$.
If $Y = X$, we say that $R$ is a relation __on__ $X$.  Such a relation
can have one or more of the following properties.

<div class="note" markdown="1">
* (R) $R$ is __reflexive__ if $xRx$ for all $x \in X$;
* (S) $R$ is __symmetric__ if $xRy$ implies $yRx$ for all $x, y \in X$;
* (T) $R$ is __transitive__ if $xRy$ and $yRz$ imply that $xRz$ for all $x, y, z \in X$;

* (I) $R$ is __irreflexive__ if not $xRx$ for all $x \in X$;
* (A) $R$ is __antisymmetric__ if $xRy$ and $yRx$ imply that
$x = y$ for all $x, y \in X$.
</div>

A relation that is (RST), i.e., reflexive, symmetric and transitive, is
called an __equivalence relation__, and it partitions the set $X$ into
(mutually disjoint) parts, called __equivalence classes__.  A relation
that is (RAT) is called a __partial order__ (such as the *divides*
partial order on the natural numbers, or the *contains* relation
between the subsets of a set).

In view of these notions, we can now describe simple graphs and some
of their properties
as follows: A *simple* graph with node set $X$ is a *symmetric*,
*irreflexive* relation on $X$.  A *directed* graph with node set $X$
is *irreflexive* if it has *no loops*.  And it is *antisymmetric* if
every edge has a *unique direction*.

The article [Counting Transitive Relations] (in the *Journal of
Integer Sequences*) contains a systematic account on the various types
of relations that can be distinguished by these 5 properties, and a
discussion of how to count them (up to equivalence) in case the
underlying set $X$ is finite.

## Composition

Relations can be composed, like functions.  If $R$ is a relation
from a set $X$ to a set $Y$, and if $S$ is a relation from $Y$ to a set $Z$,
then the __composite relation__ $R \circ S$ is the relation
from $X$ to $Z$, defined by $x (R \circ S) z$ if there is a
an element $y \in Y$ such that $x R y$ and $y S z$.

The adjacency matrix of the composite relation $R \circ S$
is essentially the (matrix) product of the adjacency matrices
of the individual relations $R$ and $S$.
If $A = (a_{ij})$ is the adjacency matrix of $R$, and $B = (b_{jk})$ the adjacency matrix of $S$,
then the $i,k$-entry of the product $AB$ is

$$(AB)_{ik} = \sum_{j} a_{ij} b_{jk}$$,

which is exactly the _number_ of elements $y \in Y$ such that $x_i R
y$ and $y S z_k$.  All it needs for $x_i$ to be $(R \circ S)$-related
to $z_k$ is this number to be at least $1$.  Hence, replacing all
nonzero entries in the product matrix $AB$ with $1$ yields the
adjacency matrix of the composite $R \circ S$.

Let's write $A \circ B$ for this matrix.

[graph1]: /images/graph1.png
[components]: /images/components.png
[bipartite1]: /images/bipartite1.png
[bipartite2]: /images/bipartite2.png
[graph2]: /images/graph2.png
[counting transitive relations]: https://cs.uwaterloo.ca/journals/JIS/VOL7/Pfeiffer/pfeiffer6.html

##  Code Corner

### `python`

* `**opts`: `python` function calls take **positional** arguments and **keyword** arguments.  
  The keyword arguments can be collected in a dictionary `opts` (with the keywords as keys).  
  This dictionary can then be passed into the function call in its "unwrapped" form `**opts`.

* `list` [doc] turns its argument into a `python` list (if possible).

In [ ]:
list("networks")

### `networkx`

* the `read_adjlist` command [doc] constructs a graph from a text file in `adj` format.

* `G.nodes` [doc] returns the nodes of a graph `G` (as an iterator).

* `G.edges` [doc] returns the edges of a graph `G` (as an iterator).

## `matplotlib`

* `subplot`: [[doc]](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html)